![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FMLOps&file=Vertex+AI+Pipelines+-+Introduction.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/MLOps/Vertex%20AI%20Pipelines%20-%20Introduction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FMLOps%2FVertex%2520AI%2520Pipelines%2520-%2520Introduction.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/MLOps/Vertex%20AI%20Pipelines%20-%20Introduction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/MLOps/Vertex%20AI%20Pipelines%20-%20Introduction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

---
This is part a [series of notebook based workflows](./readme.md) that teach all the ways to use pipelines and experiments within Vertex AI. The suggested order and description/reason is:

||Notebook Workflow|Description|
|---|---|---|
|_**This Notebook**_|[Vertex AI Pipelines - Introduction](./Vertex%20AI%20Pipelines%20-%20Introduction.ipynb)|Introduction to pipelines with the console and Vertex AI SDK|
||[Vertex AI Pipelines - Components](./Vertex%20AI%20Pipelines%20-%20Components.ipynb)|An introduction to all the ways to create pipeline components from your code|
||[Vertex AI Pipelines - IO](./Vertex%20AI%20Pipelines%20-%20IO.ipynb)|An overview of all the type of inputs and outputs for pipeline components|
||[Vertex AI Pipelines - Control](./Vertex%20AI%20Pipelines%20-%20Control.ipynb)|An overview of controlling the flow of exectution for pipelines|
||[Vertex AI Pipelines - Secret Manager](./Vertex%20AI%20Pipelines%20-%20Secret%20Manager.ipynb)|How to pass sensitive information to pipelines and components|
||[Vertex AI Pipelines - Scheduling](./Vertex%20AI%20Pipelines%20-%20Scheduling.ipynb)|How to schedule pipeline execution|
||[Vertex AI Pipelines - Management](./Vertex%20AI%20Pipelines%20-%20Management.ipynb)|Managing, Reusing, and Storing pipelines and components|
||[Vertex AI Experiments](./Vertex%20AI%20Experiments.ipynb)|Understanding and using Vertex AI Experiments|

To discover these notebooks as part of an introduction to MLOps [start here](./readme.md)!

---

# Vertex AI Piplines - Introduction

When an ML workflow has more than one step it can benefit from a pipeline.  [Vertex AI Pipelines]() is a managed service that executes [Kubeflow Pipelines (KFP)](https://www.kubeflow.org/docs/components/pipelines/v2/introduction/) and [TensorFlow Extended (TFX)](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) pipelines.  For this introduction the focus will be on KFP due to its vast flexibility, it was even originally developed as a simplified way of running TFX on Kubernetes - [history of Kubeflow](https://www.kubeflow.org/docs/started/introduction/#history).

This notebook based workflow will introduce KFP pipelines runninng on Vertex AI Pipelines and point to the additional detailed workflows within [this repository](./readme.md) that help with a deeper understanding.

---

**Pipelines** - The Concept

Pipelines are constructed from steps.  The steps are called **tasks** and utilize **components**.

The tasks are connected by inputs and outputs.  This forms the execution graph of the pipeline.

Inputs and outputs can be made up of **parameters** (`str`, `int`, `float`, `bool`, `dict`, `list`) as well as **artifacts**.  Artifacts are multiparameter objects that have defined schemas for machine learning objects (datasets, models, etc.) and automatically get stored as [Vertex AI ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction) and have lineage.

**Kubeflow Pipelines** - Constructing Pipelines

Pipelines are written in Python using the [`kfp` package](https://www.kubeflow.org/docs/components/pipelines/v2/installation/).  This SDK has decorators for multiple ways of writing [**components**](https://www.kubeflow.org/docs/components/pipelines/v2/components/) and constructing [**pipelines**](https://www.kubeflow.org/docs/components/pipelines/v2/pipelines/) from the components with features to control the execution and flow of the pipeline. Pipeline code is [**compiled**](https://www.kubeflow.org/docs/components/pipelines/v2/compile-a-pipeline/) into a YAML file using a single line command with the SDK.  

**Vertex AI Pipelines** - Executing Pipelines

The resulting YAML file is the input for [running the pipeline](https://cloud.google.com/vertex-ai/docs/pipelines/run-pipeline) with Vertex AI Pipelines.  The runs can even be scheduled with [Vertex AI Pipeline scheduler API](https://cloud.google.com/vertex-ai/docs/pipelines/schedule-pipeline-run).

The runs of a pipeline can be directly monitored, and compared, in the Vertex AI console:

**Pipeline Dashboard View In The Console: Overall Pipeline**
<p><center>
    <img src="attachment:a3351e52-b714-4434-a798-b6385edfe088.png" width="75%">
</center><p>

---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [61]:
PROJECT_ID = 'audacy-demo' # replace with project ID

In [62]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [63]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.51.0'),
    ('kfp', 'kfp'),
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        %pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            %pip install {package[1]} -U -q --user

## API Enablement

In [64]:
!gcloud auth application-default set-quota-project audacy-demos
!gcloud config set project audacy-demos
!gcloud services enable aiplatform.googleapis.com


Credentials saved to file: [/home/user/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "audacy-demos" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.
Updated property [core/project].


### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [65]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

Inputs

In [66]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'audacy-demos'

In [67]:
REGION = 'us-central1'
EXPERIMENT = 'pipeline-intro'
SERIES = 'mlops'

# gcs bucket
GCS_BUCKET = PROJECT_ID

Packages

In [68]:
import os
import time
import importlib
from google.cloud import aiplatform
import kfp
from typing import NamedTuple

In [69]:
kfp.__version__

'2.7.0'

In [70]:
aiplatform.__version__

'1.52.0'

Clients

In [71]:
# vertex ai clients
aiplatform.init(project = PROJECT_ID, location = REGION)

parameters:

In [72]:
DIR = f"temp/{SERIES}-{EXPERIMENT}"

In [73]:
# SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
# SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT = "mlops-service-account@audacy-demos.iam.gserviceaccount.com"

environment:

In [74]:
if not os.path.exists(DIR):
    os.makedirs(DIR)

---
## KFP Pipeline On Vertex AI Pipelines

An example workflows: a KFP pipeline constructed from custom components and run on Vertex AI Pipelines.

### Create Pipeline Components

These are simple Python components, specifically lightweight Python components.  

Simple component with multiple inputs of different data types and outputs a dictionary.

In [75]:
@kfp.dsl.component(
    base_image = "python:3.11",
    packages_to_install = ["pandas"]
)
def example_parameters(
    in_str: str,
    in_int: int,
    in_float: float,
    in_bool: bool,
    in_list: list
) -> dict:
    results = dict(
        ex_str = in_str,
        ex_int = in_int,
        ex_float = in_float,
        ex_bool = in_bool,
        ex_list = in_list
    )
    return results

Simple component that take input parameters and outputs an artifact.

In [76]:
@kfp.dsl.component(
    base_image = "python:3.10",
    packages_to_install = ["pandas"]
)
def example_artifact(
    metadata: dict
) -> kfp.dsl.Artifact:
    ex_artifact = kfp.dsl.Artifact(
        metadata = metadata,
        uri = 'https://www.kubeflow.org/docs/components/pipelines/v2/data-types/artifacts/#artifact-types'
    )
    return ex_artifact

### Create Pipeline

In [77]:
@kfp.dsl.pipeline(
    name = f'{SERIES}-{EXPERIMENT}',
    description = 'An introduction pipeline',
    pipeline_root = f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/pipeline_root'
)
def intro_pipeline(
    in_str: str,
    in_int: int,
    in_float: float,
    in_bool: bool,
    in_list: list
) -> dict:
    
    task1 = example_parameters(
        in_str = in_str,
        in_int = in_int,
        in_float = in_float,
        in_bool = in_bool,
        in_list = in_list
    )
    
    with kfp.dsl.If(in_bool == True, name = 'Proceed If True'):
        task2 = example_artifact(metadata = task1.output)

    return task1.output

### Compile Pipeline

In [78]:
kfp.compiler.Compiler().compile(
    pipeline_func = intro_pipeline,
    package_path = f'{DIR}/{SERIES}-{EXPERIMENT}.yaml'
)

### Create Pipeline Job (With Vertex AI SDK)

The compiled pipeline file can be submitted for running with the console or the SDK (shown here).  Check out the details in the documentation [here](https://cloud.google.com/vertex-ai/docs/pipelines/run-pipeline#create_a_pipeline_run) for an overview with the console.


In [79]:
pipeline_job = aiplatform.PipelineJob(
    display_name = f"{SERIES}-{EXPERIMENT}",
    template_path = f"{DIR}/{SERIES}-{EXPERIMENT}.yaml",
    parameter_values = dict(
        in_str = 'An Example String',
        in_int = 45,
        in_float = 4.5,
        in_bool = True,
        in_list = [1, 27, 'Another String']
    ),
    pipeline_root = f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/pipeline_root',
    enable_caching = None # True (enabled), False (disable), None (defer to component level caching) 
)

### Submit Pipeline Job (On Vertex AI Pipelines)

In [80]:
response = pipeline_job.submit(
    service_account = SERVICE_ACCOUNT
)

Creating PipelineJob
PipelineJob created. Resource name: projects/988062960808/locations/us-central1/pipelineJobs/mlops-pipeline-intro-20240521232257
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/988062960808/locations/us-central1/pipelineJobs/mlops-pipeline-intro-20240521232257')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/mlops-pipeline-intro-20240521232257?project=988062960808


In [81]:
print(f'The Dashboard can be viewed here:\n{pipeline_job._dashboard_uri()}')

The Dashboard can be viewed here:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/mlops-pipeline-intro-20240521232257?project=988062960808


In [82]:
pipeline_job.wait()

PipelineJob run completed. Resource name: projects/988062960808/locations/us-central1/pipelineJobs/mlops-pipeline-intro-20240521232257


---
## Results: Vertex AI Console And SDK

As soon as the job is submitted the response gives a link to the the dashboard view in the console for Vertex AI Pipelines.  Using this link, or navigating directly though Vertex AI, gives a dashboard view of the pipeline as it is running with progress indicatiors for each task.  This information call also be retrieved using the Vertex AI SDK. Both approaches will be covered in this section for:
- Pipeline Run
- Pipeline Run Tasks Level
- Pipeline IO Artifacts
- Artifact Lineage In Vertex AI ML Metadata

### Pipeline Run Overview

**Pipeline Dashboard View In The Console: Overall Pipeline**
<p><center>
    <img src="attachment:e73f5af8-ff4e-4c6c-b785-02b00ed9cfc9.png" width="75%">
</center><p>

Note that:
- each node, or tasks, is displayed.
- the connections between nodes are visual and reflect the input/output dependencies
- the node purpose is reflected with an icon and the status is reflected with an icon and color coding
- a summary of the pipeline run is provide on the right and includes all the pipelines inputs parameters.

    
The same information can be retrieved with the Vertex AI SDK as follows.
- SDK Reference: [`aiplatform.PipelineJob`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.PipelineJob)

#### Retrieve Latest Run: By Display Name

In [83]:
aiplatform.PipelineJob.list(
    project = PROJECT_ID,
    location = REGION,
    filter = f'display_name="{SERIES}-{EXPERIMENT}"',
    order_by='update_time desc'
)[0]

resource name: projects/988062960808/locations/us-central1/pipelineJobs/mlops-pipeline-intro-20240521232257

#### Retrieve all runs to dataframe:
- SDK Refrence: [`aiplatform.get_pipeline_df`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_get_pipeline_df)

In [84]:
aiplatform.get_pipeline_df(pipeline = f'{SERIES}-{EXPERIMENT}')

,pipeline_name,run_name,param.input:in_bool,param.input:in_list,param.input:in_str,param.output:Output,param.input:in_int,param.vmlmd_lineage_integration,param.input:in_float
0,mlops-pipeline-intro,mlops-pipeline-intro-20240521232257,True,"[1.0, 27.0, Another String]",An Example String,"{'ex_bool': True, 'ex_float': 4.5, 'ex_str': '...",45.0,{'pipeline_run_component': {'parent_task_names...,4.5
1,mlops-pipeline-intro,mlops-pipeline-intro-20240521231829,True,"[1.0, 27.0, Another String]",An Example String,"{'ex_float': 4.5, 'ex_bool': True, 'ex_str': '...",45.0,{'pipeline_run_component': {'parent_task_names...,4.5
2,mlops-pipeline-intro,mlops-pipeline-intro-20240521223005,True,"[1.0, 27.0, Another String]",An Example String,"{'ex_bool': True, 'ex_float': 4.5, 'ex_str': '...",45.0,{'pipeline_run_component': {'parent_task_names...,4.5


#### Review PipelineJob Object

In [85]:
pipeline_job

resource name: projects/988062960808/locations/us-central1/pipelineJobs/mlops-pipeline-intro-20240521232257

In [86]:
type(pipeline_job)

google.cloud.aiplatform.pipeline_jobs.PipelineJob

In [87]:
pipeline_job.state

<PipelineState.PIPELINE_STATE_SUCCEEDED: 4>

In [88]:
pipeline_job.display_name

'mlops-pipeline-intro'

In [89]:
pipeline_job.name

'mlops-pipeline-intro-20240521232257'

In [90]:
pipeline_job.to_dict().keys()

dict_keys(['name', 'displayName', 'createTime', 'startTime', 'endTime', 'updateTime', 'pipelineSpec', 'state', 'jobDetail', 'labels', 'runtimeConfig', 'serviceAccount'])

In [91]:
pipeline_job.to_dict()['pipelineSpec'].keys()

dict_keys(['deploymentSpec', 'components', 'root', 'schemaVersion', 'sdkVersion', 'deploymentConfig', 'pipelineInfo', 'defaultPipelineRoot'])

### Pipeline Run: Task Overview

Each component used in the pipeline created a task.  These task can be retrieved, reviewed, and even examine for input/ouput values of parameters.

**Pipeline Dashboard View In The Console: Task Level Details**
<p><center>
    <img src="attachment:dbed9d44-8308-421b-bc88-993badc60bd1.png" width="75%">
</center><p>

#### Review Tasks with `PipelineJob.task_details`

In [92]:
[(task.task_name, task.state) for task in pipeline_job.task_details]

[('example-parameters', <State.SKIPPED: 8>),
 ('mlops-pipeline-intro-20240521232257', <State.SUCCEEDED: 3>),
 ('example-artifact', <State.SKIPPED: 8>),
 ('condition-1', <State.SUCCEEDED: 3>)]

In [93]:
tasks = {task.task_name: task for task in pipeline_job.task_details}

In [94]:
#tasks['example-parameters'].execution.metadata

#### Review Tasks With `PipelineJob.gca_resource` As Dictionary with `.to_dict()`

In [95]:
pipeline_job.to_dict()['jobDetail']['taskDetails'][1]['execution']['metadata']

{'input:in_float': 4.5,
 'input:in_list': [1.0, 27.0, 'Another String'],
 'input:in_str': 'An Example String',
 'output:Output': {'ex_float': 4.5,
  'ex_bool': True,
  'ex_str': 'An Example String',
  'ex_list': [1.0, 27.0, 'Another String'],
  'ex_int': 45.0},
 'input:in_int': 45.0,
 'vmlmd_lineage_integration': {'pipeline_run_component': {'parent_task_names': [],
   'project_id': 'audacy-demos',
   'pipeline_run_id': 'mlops-pipeline-intro-20240521232257',
   'location_id': 'us-central1',
   'task_name': 'mlops-pipeline-intro-20240521232257'}},
 'input:in_bool': True}

### Pipeline Run: `CustomJob` For Task:

Each task runs as a Vertex AI Training Job.  The details of these jobs can also be reviewed directly from the console by click the 'VIEW JOB' link in the task overview panel.  This opens the exact training job under Vertex AI Training:

**Vertex AI Training Job In The Console: Task Level Job Resources**
<p><center>
    <img src="attachment:9d3dc27b-6cd0-44cc-88ab-b3ba2dd60b74.png" width="75%">
</center><p>

### Retrieve `CustomJob` Details Using `PipelineJob` object:

In [100]:
task_job_id = pipeline_job.to_dict()['jobDetail']['taskDetails'][0]['executorDetail']['containerDetail']['mainJob']
task_job_id

'projects/988062960808/locations/us-central1/customJobs/7430076852649066496'

In [101]:
task_job = aiplatform.CustomJob.get(resource_name = task_job_id)

In [102]:
type(task_job)

google.cloud.aiplatform.jobs.CustomJob

In [103]:
task_job

resource name: projects/988062960808/locations/us-central1/customJobs/7430076852649066496

In [104]:
task_job.display_name

'caip_pipelines_2079287813676728320_-8260918587930181632'

In [105]:
task_job.to_dict()

{'name': 'projects/988062960808/locations/us-central1/customJobs/7430076852649066496',
 'displayName': 'caip_pipelines_2079287813676728320_-8260918587930181632',
 'jobSpec': {'workerPoolSpecs': [{'machineSpec': {'machineType': 'e2-standard-4'},
    'replicaCount': '1',
    'diskSpec': {'bootDiskType': 'pd-ssd', 'bootDiskSizeGb': 100},
    'containerSpec': {'imageUri': 'python:3.11',
     'command': ['sh',
      '-c',
      '\nif ! [ -x "$(command -v pip)" ]; then\n    python3 -m ensurepip || python3 -m ensurepip --user || apt-get install python3-pip\nfi\n\nPIP_DISABLE_PIP_VERSION_CHECK=1 python3 -m pip install --quiet --no-warn-script-location \'kfp==2.7.0\' \'--no-deps\' \'typing-extensions>=3.7.4,<5; python_version<"3.9"\'  &&  python3 -m pip install --quiet --no-warn-script-location \'pandas\' && "$0" "$@"\n',
      'sh',
      '-ec',
      'program_path=$(mktemp -d)\n\nprintf "%s" "$0" > "$program_path/ephemeral_component.py"\n_KFP_RUNTIME=true python3 -m kfp.dsl.executor_main     

### Pipeline IO Artifacts

The pipeline task created from the `example_artifact` component returned an artifact (`kfp.dsl.Artifact`) as output.  Pipelines automatically store artifacts in Vertex AI ML Metadata.

**Pipeline Dashboard View In The Console: Artifact Info**
<p><center>
    <img src="attachment:5eea8a64-afe7-484b-9327-503674586cd4.png" width="75%">
</center><p>

#### Retrieve Artifact Info From `PipelineJob`

In [106]:
pipeline_job.to_dict()['jobDetail']['taskDetails'][2]['execution']['metadata']

{'vertex-ai-pipelines-artifact-argument-binding': {'output:Output': ['projects/988062960808/locations/us-central1/metadataStores/default/artifacts/7372742474447565430']},
 'vmlmd_lineage_integration': {'pipeline_run_component': {'location_id': 'us-central1',
   'project_id': 'audacy-demos',
   'pipeline_run_id': 'mlops-pipeline-intro-20240521223005',
   'parent_task_names': ['mlops-pipeline-intro-20240521223005', 'condition-1'],
   'task_name': 'example-artifact'}},
 'input:metadata': {'ex_float': 4.5,
  'ex_bool': True,
  'ex_str': 'An Example String',
  'ex_list': [1.0, 27.0, 'Another String'],
  'ex_int': 45.0}}

In [109]:
artifact_id = pipeline_job.to_dict()['jobDetail']['taskDetails'][2]['outputs']['Output']['artifacts'][0]['name']
artifact_id

'projects/988062960808/locations/us-central1/metadataStores/default/artifacts/7372742474447565430'

In [110]:
artifact = aiplatform.Artifact.get(resource_id = artifact_id)

In [111]:
artifact.uri

'https://www.kubeflow.org/docs/components/pipelines/v2/data-types/artifacts/#artifact-types'

In [112]:
artifact.metadata

{'ex_float': 4.5,
 'ex_bool': True,
 'ex_str': 'An Example String',
 'ex_list': [1.0, 27.0, 'Another String'],
 'ex_int': 45.0}

#### Retrieve Pipeline Artifacts Vertex AI SDK

- [`aiplatform.Artifact()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Artifact)

In [114]:
artifacts = aiplatform.Artifact.list(
    filter = "metadata.ex_bool.bool_value = true"
)

In [115]:
artifacts

 resource name: projects/988062960808/locations/us-central1/metadataStores/default/artifacts/7372742474447565430
 uri: https://www.kubeflow.org/docs/components/pipelines/v2/data-types/artifacts/#artifact-types
 schema_title:system.Artifact]

In [116]:
type(artifacts[0])

google.cloud.aiplatform.metadata.artifact.Artifact

In [117]:
artifacts[0].metadata

{'ex_float': 4.5,
 'ex_bool': True,
 'ex_str': 'An Example String',
 'ex_list': [1.0, 27.0, 'Another String'],
 'ex_int': 45.0}

In [118]:
artifacts[0].lineage_console_uri

'https://console.cloud.google.com/vertex-ai/locations/us-central1/metadata-stores/default/artifacts/7372742474447565430?project=988062960808'

### Pipeline IO Artifact Metadata Lineage

The consolve view of artifact info include a link to the Vertex AI ML Metadata.  This link can also be directly retrieved for the artifact using the SDK with `lineage_console_uri` attribute.  This ones the lineage view of the artifact in the Vertex AI ML Metadata Console.


**Vertex AI ML Metadata Console: Artifact Lineage**
<p><center>
    <img src="attachment:ae07a90a-c046-493f-ae29-04e12f473465.png" width="75%">
</center><p>

In [119]:
artifact.lineage_console_uri

'https://console.cloud.google.com/vertex-ai/locations/us-central1/metadata-stores/default/artifacts/7372742474447565430?project=audacy-demos'